In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load('en')
from spacy.symbols import nsubj

In [19]:
from settings import *
from utils import *

[autoreload of settings failed: Traceback (most recent call last):
  File "/Applications/anaconda/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
pandas.core.config.OptionError: "No such keys(s): 'display.max_collength'"
]


### Loads GloVe Embeddings

In [3]:
from gloveEmbeddings import *
loadGloveEmbeddings(gloveFile)

### Creates Keyword Vectors

In [4]:
def keywordVectors():
    source = ['paper', 'report', 'study', 'analysis', 'research', 'survey']
    people = ['expert', 'scientist']
    subject = source + people
    predicate = ['prove', 'demonstrate', 'reveal', 'state', 'mention', 'report', 'say', 'show', 'announce', 'claim', 'suggest']
    
    subjectVec =[word2vec(s) for s in subject]
    predicateVec = [word2vec(p) for p in predicate]
    
    return subjectVec, predicateVec

### Searches (on the vector space) for sentences containing the given subject and predicate keywords.

In [5]:
def keywordSearch(title, body, subjectVec, predicateVec):
    subjectThreshold = 0.9
    predicateThreshold = 0.9

    claims = []
    for s in sent_tokenize(body):
        subjectFound = predicateFound = False
        claim = ""
        for w in wordpunct_tokenize(s):

            if predicateFound == True:
                claim = s
                claims.append(claim)
                break

            wVec = word2vec(w)

            if subjectFound == False:
                for sVec in subjectVec:
                    if abs(cosine_similarity(sVec.reshape(1, -1), wVec.reshape(1, -1))) > subjectThreshold:
                        subjectFound = True
                        break

            if subjectFound == True:
                for pVec in predicateVec:
                    if abs(cosine_similarity(pVec.reshape(1, -1), wVec.reshape(1, -1))) > predicateThreshold:
                        predicateFound = True
                        break
    return claims


In [13]:
def claimExtraction(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)

    subjectVec, predicateVec = keywordVectors()
    
    
    
    def dependencyGraphSearch(title, body, subjectVec, predicateVec):
        
        claims = []
        for s in sent_tokenize(body):
            subjectFound = predicateFound = False
            claim = ""
            holder = ""
            
            try:
                doc = nlp(s)
                rootVerb = [w for w in doc if w.head is w][0]
            except:
                continue
            
            wVec = word2vec(str(rootVerb))
            for pVec in predicateVec:
                if abs(cosine_similarity(pVec.reshape(1, -1), wVec.reshape(1, -1))) > 0.9:
                    predicateFound = True
                    print ("sentence:",s)
                    break

                        
            if predicateFound == True:
                for np in doc.noun_chunks:
                    if(np.root.dep == nsubj and np.root.head == rootVerb):
                        wVec = word2vec(str(np.root.text))
                    
                        for sVec in subjectVec:    
                            if abs(cosine_similarity(sVec.reshape(1, -1), wVec.reshape(1, -1))) > 0.5:
                                    subjectFound = True
                                    holder = np.text
                                    break
                
            if subjectFound == True:
                    claim = s
                    claims.append(claim)
                    print('claim:', claim)
                    print('by:', holder)
                    print()
                    break

        return claims

    
    claims = documents.apply(lambda d: dependencyGraphSearch(d['title'],d['body'], subjectVec, predicateVec), axis=1)
    #return claims

claimExtraction(50)    

sentence: Three studies show the benefits of healthier school meals.
claim: Three studies show the benefits of healthier school meals.
by: Three studies

sentence: Many studies show that calorie intake is higher when people dine out.
claim: Many studies show that calorie intake is higher when people dine out.
by: Many studies

sentence: The findings, published in the June 24 online issue of Nature Neuroscience, suggest that an increased interest in food leads to a decreased interest in novelty and cocaine.
claim: The findings, published in the June 24 online issue of Nature Neuroscience, suggest that an increased interest in food leads to a decreased interest in novelty and cocaine.
by: The findings

sentence: Generally speaking, people believe that either genetics--if your mom or dad was obese, then you're likely to be as well--or personal choices are to blame.
claim: Generally speaking, people believe that either genetics--if your mom or dad was obese, then you're likely to be as wel

sentence: A new scientific study shows that something similar is true of the crops that farmers grow.
claim: A new scientific study shows that something similar is true of the crops that farmers grow.
by: A new scientific study

sentence: The Lancet analysis shows high blood pressure, smoking and drinking alcohol have become the highest risk factors for ill health.
claim: The Lancet analysis shows high blood pressure, smoking and drinking alcohol have become the highest risk factors for ill health.
by: The Lancet analysis

sentence: The 2012 America's Health Rankings show troubling levels of obesity, diabetes, high blood pressure and sedentary behavior.
sentence: Not to mention all of the McDonald's grease you're selling," El-Sayed wrote.
sentence: Studies show that if we mess with the body's natural sleep-wake cycle — say, by working an overnight shift, taking a transatlantic flight or staying up all night with a new baby or puppy — we pay the price.
claim: Studies show that if we mes

In [18]:
def annotateDataset(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)
    return documents
annotateDataset(20)

,title,body
0,"Three studies on school food: Nutrition standards work, and well","The School Nutrition Association’s bizarre opposition to the USDA’s nutrition standards for school meals has stimulated research.\n\nThree studies show the benefits of healthier school meals.\n\n1. From The Rudd Center, now at U. Conn: A press release announces publication of its new study in Childhood Obesity demonstrating that the rules have led to an increase in fruit consumption without increasing plate waste.\n\nUSDA, understandably pleased with this result, quotes Secretary Tom Vilsack in a press release:\n\nFor Congress to meddle with doctors’ recommendations and go back to less healthy meals now would not be in the best interest of our children.\n\n2. From the Harvard School of Public Health: It also sends a press release to announce its study demonstrating that an increase in consumption of fruits and vegetables is a direct result of the new USDA standards, and that these also do not increase plate waste.\n\n3. From the Union of Concerned Scientists: UCS announces a new position paper, “Lessons f..."
1,Pizza chains band together over proposed menu-labeling plan,"Domino's Pizza manager T.J. Djigue creates a hand tossed pizza in Bethesda, MD. The FDA is crafting regulations that would require major chain restaurants to label calorie counts on their menus. (Katherine Frey/THE WASHINGTON POST)\n\nA Big Mac is a Big Mac.\n\nBut a pizza? There are 34 million ways to customize it, depending on toppings and crust and size, according to the number-crunchers at Domino’s.\n\nThat’s the message that some of the nation’s largest pizza chains want to impress upon Capitol Hill lawmakers Wednesday as they push for changes to a proposed menu labeling plan that would force chain restaurants and other food outlets to post the calorie counts of the foods they’re selling.\n\nA fledgling coalition of pizza chains — including Domino’s, Papa John’s, Little Caesars, Godfather’s Pizza and Pizza Hut — argues that the government’s plan forces store owners to pay for in-store menu boards that most of their customers don’t see before ordering. The American Pizza Community, as the coalition is..."
2,"Overeating And Cocaine Abuse Controlled By Same Neurons In Brain, Study Says","People less interested in food might be at a higher risk for cocaine addiction, according to researchers at Yale School of Medicine.\n\nThe new study found that a set of neurons located in the part of the brain that controls hunger are associated not only with overeating, but other behaviors not linked to food, like novelty-seeking and drug addiction.\n\nThe findings, published in the June 24 online issue of Nature Neuroscience, suggest that an increased interest in food leads to a decreased interest in novelty and cocaine.\n\nYaleNews has more information from the one of the study's leaders, Tamas L. Horvath, the Jean and David W. Wallace Professor of Biomedical Research and chair of comparative medicine at Yale School of Medicine:\n\nHorvath and his team argue that the hypothalamus, which controls vital functions such as body temperature, hunger, thirst fatigue and sleep, is key to the development of higher brain functions. ""These hunger-promoting neurons are critically important during development to e..."
3,Report Suggests Social Environment's Outsized Role in Obesity,"Obesity is a global epidemic. The World Health Organization now places it as the world's leading non-communicable disease, ahead of even diabetes or cancer. And Americans are more obese than most.\n\nGenerally speaking, people believe that either genetics--if your mom or dad was obese, then you're likely to be as well--or personal choices are to blame. Most obesity researchers--and most diets--focus nearly exclusively on the individual and the daily food choices each of us makes.\n\nNew, fad diets or efforts to limit what we eat and drink make news constantly. New York Mayor Michael Bloomberg's efforts to limit the serving s